In [1]:
import numpy as np
import pygame
from typing import Tuple
import time
from collections import deque
import sys
import os
import pickle


def normalize_radians(radians):
    normalized_radians = radians % (2 * np.pi)  
    if normalized_radians > np.pi:
        normalized_radians -= 2 * np.pi
    return normalized_radians

pygame 2.5.2 (SDL 2.28.3, Python 3.12.1)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
WIDTH, HEIGHT = 800, 600
WHITE = (255, 255, 255)
RED = (255, 0, 0)
STARTINGPOSITION = (100, 300)
DOTPERMETER = 15
EPSILON = 0.2
CARLENGTH = 5
CARWIDTH = .423
SPEED = 6.94*1.5 # m/s -> 25*X km/h

GRID_SIZE_SMALL = 30    #small
GRID_SIZE_MEDIUM = 15   #medium
GRID_SIZE_HIGH = 7      #high
GRID_SIZE_VERYHIGH = 3  #ultra high
GRID_SIZE = 30          #default

FPS = 60  # Frames per second for the simulation
CAR_COLOR = (0, 255, 0)  # Green car
OBSTACLE_COLOR = (255, 0, 0)  # Red obstacles
PATH_COLOR = (0, 0, 255)  # Blue path
GOAL_COLOR = (0, 255, 255)  # Cyan goal

HOVER_COLOR = (150, 150, 150)  # Hover color (slightly darker)
CLICK_COLOR = (100, 100, 100)  # Clicked color (even darker)

carImage = pygame.image.load("media/rect.png")
carImage = pygame.transform.scale(surface= carImage, size=[ CARLENGTH*DOTPERMETER, CARWIDTH*DOTPERMETER])
wheelImage = pygame.image.load("media/wheel.png")
wheelImage = pygame.transform.scale(surface= wheelImage, size=[ CARLENGTH*DOTPERMETER/3, CARWIDTH*DOTPERMETER*2])
bfsImage = pygame.image.load("media/bfs.png")
dfsImage = pygame.image.load("media/dfs.png")
astarImage = pygame.image.load("media/astar.png")


In [3]:
class Car:
    goal = (-1,-1)
    def __init__(self, maxSteeringAngle : float, position : Tuple[float, float] , maxSpeed : float, headingAngle : float, length : float):
        """ angles in radians """
        self.__maxSteeringAngle = maxSteeringAngle
        """in radian"""
        self.__position = position
        self.__maxSpeed = maxSpeed
        self.__headingAngle = headingAngle
        """theta, in radian"""

        
        self.__steeringRate = 0

        self.__steeringAngle = 0.0
        """gamma, in radian"""
        self.__length = length
        self.__speed = 0

    def get_max_speed(self):
        return self.__maxSpeed
    
    def get_position(self) -> Tuple[float, float]:
        return self.__position
    
    def set_position(self, position : Tuple[float, float]):
        self.__position = position

    def set_speed_rate(self, rate : float):
        """between 1 (max speed forward) and -1 (max speed backward)"""
        self.__speed = self.__maxSpeed * rate

    def get_speed(self):
        return self.__speed

    def get_heading_angle(self):
        """in radian"""
        return self.__headingAngle
    
    def get_max_steering_angle(self):
        """in radian"""
        return self.__maxSteeringAngle

    def set_steering_rate(self, rate : float):
        """between 1 (fully left) and -1 (fully right)"""
        self.__steeringAngle = self.__maxSteeringAngle * rate
        self.__steeringRate = rate

    def get_steering_rate(self):
        return self.__steeringRate

    def get_steering_angle(self):
        """in radian"""
        return self.__steeringAngle
    
    def get_car_length(self):
        return self.__length

    def travel(self, dt : float):
        beta = np.arctan(self.__length/2 * np.tan(self.__steeringAngle) / self.__length)
        posUpdate = (np.cos(beta+self.__headingAngle),np.sin(beta+self.__headingAngle))
        self.__position = np.add(self.__position, np.multiply((posUpdate[0], -posUpdate[1]), self.__speed * dt * DOTPERMETER))
        self.__headingAngle = normalize_radians(self.__headingAngle + (self.__speed*np.tan(self.__steeringAngle) *np.cos(beta) / self.__length)*dt)

    #this will get 4 evenly spaced points around the car
    #the car is a line segment
    def get_Five_Points(self):
        headingAngle = car.get_heading_angle()
        carPosition = car.get_position()
        headingDir = (np.cos(headingAngle), -np.sin(headingAngle))
        p1 = np.add(carPosition, np.multiply(headingDir, car.get_car_length()/4*DOTPERMETER))
        p2 = np.add(carPosition, np.multiply(headingDir, car.get_car_length()/2*DOTPERMETER))
        p3 = np.add(carPosition, np.multiply(headingDir, -car.get_car_length()/4*DOTPERMETER))
        p4 = np.add(carPosition, np.multiply(headingDir, -car.get_car_length()/2*DOTPERMETER))
        p5 = carPosition
        return [p1, p2, p3, p4, p5]

    def get_N_Times_2_plus_1_Points(self, n: int):
        headingAngle = car.get_heading_angle()
        carPosition = car.get_position()
        headingDir = (np.cos(headingAngle), -np.sin(headingAngle))
        points = []
        for i in range(n):
            points.append(np.add(carPosition, np.multiply(headingDir, car.get_car_length()/2*DOTPERMETER*(i+1)/n)))
            points.append(np.add(carPosition, np.multiply(headingDir, -car.get_car_length()/2*DOTPERMETER*(i+1)/n)))
        points.append(carPosition)
        return points
    
    def get_minimum_turning_radius(self):
        return self.get_car_length()/np.tan(self.get_max_steering_angle())

#instantiation
car = Car(maxSteeringAngle=np.deg2rad(40),
          position=STARTINGPOSITION,
          maxSpeed=SPEED,
          headingAngle=0.0,
          length= CARLENGTH) #5 meters long
goal = (-1,-1)


### BFS

In [4]:
class BFS:
    def __init__(self, grid):
        self.grid = grid
        self.directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]
        self.visited = [[False for _ in range(len(grid[0]))] for _ in range(len(grid))]

    
    def bfs_path(self, start, goal, grid):
        
        self.grid = grid
        queue = deque([(start, [start])])  # Queue holds (current cell, path to current cell)
        visited = set()
        visited.add(start)

        while queue:
            (x, y), path = queue.popleft()

            # Check if we've reached the target
            if (x, y) == goal:
                return path

            # Explore each direction
            for dx, dy in self.directions:
                new_x, new_y = int(x + dx), int(y + dy)
                new_cell = (new_x, new_y)
                if 0 <= new_x < len(self.grid) and 0 <= new_y < len(self.grid[0]) and \
                   self.grid[new_x][new_y] != 1 and new_cell not in visited:
                    queue.append((new_cell, path + [new_cell]))
                    visited.add(new_cell)

        return None  # No path found


### DFS

In [5]:
class DFS:
    def __init__(self, grid):
        self.grid = grid
        self.directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]
        self.visited = [[False for _ in range(len(grid[0]))] for _ in range(len(grid))]

    def dfs_path(self, start, goal, grid):

        self.visited = [[False for _ in range(len(grid[0]))] for _ in range(len(grid))]
        self.grid = grid
        path = []
        return self._dfs(start, goal, path)

    def _dfs(self, start, target, path):
        # If the cell is out of bounds, an obstacle, or already visited, return None
        (x,y) = start
        (x,y) = (int(x),int(y))
        if x < 0 or x >= len(self.grid) or y < 0 or y >= len(self.grid[0]) or \
           self.visited[x][y] or self.grid[x][y] == 1:
            return None

        # Add the current cell to the path
        path.append((x, y))

        # Check if we've reached the target
        if (x, y) == target:
            return path

        # Mark this cell as visited
        self.visited[x][y] = True

        # Explore each direction
        for dx, dy in self.directions:
            new_x, new_y = int(x + dx), int(y + dy)
            result = self._dfs((new_x, new_y), target, path)
            if result:  # Path found
                return result

        # Backtrack if no path is found in this direction
        path.pop()
        return None

### A*  

In [6]:
import heapq
import math

# Define a simple Node class to store position, cost, and parent information
class Node:
    def __init__(self, position, g_cost=0, h_cost=0, parent=None):
        self.position = position  # (x, y) tuple
        self.g_cost = g_cost  # Cost from start to this node
        self.h_cost = h_cost  # Heuristic cost from this node to the goal
        self.parent = parent  # Pointer to parent node

    def f_cost(self):
        return self.g_cost + self.h_cost

    # Comparison operators for heapq
    def __lt__(self, other):
        return self.f_cost() < other.f_cost()

def a_star(start, goal, grid):
    """A* algorithm to find the shortest path on a grid."""
    # Initialize the open and closed lists
    open_list = []
    closed_list = set()

    # Create the start node and goal node
    start_node = Node(start, g_cost=0, h_cost=heuristic(start, goal))
    goal_node = Node(goal)

    # Push the start node onto the open list
    heapq.heappush(open_list, start_node)

    # Define the possible movement directions (4 or 8 directions)
    # Here we use 4-connected grid (up, down, left, right)
    directions = [(0, -1), (0, 1), (-1, 0), (1, 0)]  # No diagonals
    # Uncomment below for 8-connected grid (including diagonals)
    # directions = [(0, -1), (0, 1), (-1, 0), (1, 0), (-1, -1), (-1, 1), (1, -1), (1, 1)]

    # A dictionary to store the best g_cost for each position
    g_costs = {start: 0}

    while open_list:
        # Get the node with the lowest f_cost from the open list
        current_node = heapq.heappop(open_list)

        # Check if we've reached the goal
        if current_node.position == goal:
            return reconstruct_path(current_node)

        # Add the current node to the closed list
        closed_list.add(current_node.position)

        # Explore the neighbors
        for direction in directions:
            neighbor_pos = (current_node.position[0] + direction[0],
                            current_node.position[1] + direction[1])

            # Skip if the neighbor is out of bounds or an obstacle
            neighbor_pos = (int(neighbor_pos[0]), int(neighbor_pos[1]))

            if not valid_position(neighbor_pos, grid) or grid[neighbor_pos[0]][neighbor_pos[1]] == 1:
                continue

            # Calculate the g_cost for this neighbor
            tentative_g_cost = current_node.g_cost + 1  # Assuming cost of 1 for each move

            # If the neighbor is in the closed list, skip it
            if neighbor_pos in closed_list:
                continue

            # If this path to the neighbor is better, or it's not in the open list
            if neighbor_pos not in g_costs or tentative_g_cost < g_costs[neighbor_pos]:
                g_costs[neighbor_pos] = tentative_g_cost
                h_cost = heuristic(neighbor_pos, goal)
                neighbor_node = Node(neighbor_pos, g_cost=tentative_g_cost, h_cost=h_cost, parent=current_node)

                # Push neighbor onto the open list
                heapq.heappush(open_list, neighbor_node)

    return None  # No path found

def heuristic(position, goal):
    """Heuristic function (Manhattan distance) between two points."""
    return abs(position[0] - goal[0]) + abs(position[1] - goal[1])

def valid_position(pos, grid):
    """Check if a position is within bounds and not an obstacle."""
    x, y = pos
    return 0 <= x < len(grid) and 0 <= y < len(grid[0])

def reconstruct_path(node):
    """Reconstruct the path from goal to start by following the parent links."""
    path = []
    while node:
        path.append(node.position)
        node = node.parent
    return path[::-1]  # Reverse the path


In [7]:
# Pygame initialization
pygame.init()

goal = (-1,-1)
# Define the screen
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Path Planning Visualization")

# Clock to control the frame rate
clock = pygame.time.Clock()

# Your Car class implementation (unchanged)

# Create a car object
car = Car(
    maxSteeringAngle=np.deg2rad(40),
    position=STARTINGPOSITION,  # Starting position in the middle of the screen
    maxSpeed=SPEED,  # Max speed in pixels per second
    headingAngle=0.0,  # Facing right
    length=CARLENGTH  # Car length in meters
)

# Example grid and obstacles

def refreshGrid(size):
    GRID_SIZE = size
    grid_width = WIDTH // GRID_SIZE
    grid_height = HEIGHT // GRID_SIZE
    grid = np.zeros((grid_width, grid_height))  # All cells free
    return grid, grid_width, grid_height, GRID_SIZE

grid, grid_width, grid_height, GRID_SIZE = refreshGrid(GRID_SIZE)
# Adding obstacles in the grid
#grid[10:15, 10] = 1  # Vertical line of obstacles
#grid[19, 5:15] = 1  # Horizontal line of obstacles

# Path planning (example path)
path = [(100, 300), (200, 300), (300, 350), (400, 350), (500, 400)]

def translate_position(vector : Tuple[float, float], angle : float, length:float):
    """angle in radian"""
    newX = vector[0] + length * np.cos(angle)
    newY = vector[1] - length * np.sin(angle)

    return (newX, newY)


def draw_grid(goal):
    """Draws the grid and handles hover/click interactions."""
    mouse_x, mouse_y = pygame.mouse.get_pos()  # Get mouse position
    mouse_grid_x, mouse_grid_y = mouse_x // GRID_SIZE, mouse_y // GRID_SIZE  # Get the grid cell under the mouse

    #set mouse grid x,y not to exceed the grid
    mouse_grid_x = max(0, min(mouse_grid_x, grid_width-1))
    mouse_grid_y = max(0, min(mouse_grid_y, grid_height-1))

    for x in range(grid_width):
        for y in range(grid_height):
            rect = pygame.Rect(x * GRID_SIZE, y * GRID_SIZE, GRID_SIZE, GRID_SIZE)

            # If the mouse is hovering over this cell
            if x == mouse_grid_x and y == mouse_grid_y:
                if pygame.mouse.get_pressed()[0]:  # If the left mouse button is pressed
                    color = CLICK_COLOR  # Make the cell darker when clicked
                    grid[x, y] = 1  # Mark the cell as clicked/obstacle
                if pygame.mouse.get_pressed()[1]:  # If the middle mouse button is pressed
                    # Reset the cell to empty
                    color = (255, 255, 255)
                    grid[x, y] = 0
                elif pygame.mouse.get_pressed()[2]:  # If the right mouse button is pressed
                    color = CLICK_COLOR
                    if (goal != (-1,-1)):
                        grid[goal[0], goal[1]] = 0
                    grid[x, y] = 2
                    goal = (x,y)
                else:
                    color = HOVER_COLOR
                    if grid[x, y] == 2:
                        color = GOAL_COLOR
                    elif grid[x, y] == 1:
                        color = OBSTACLE_COLOR
            else:
                color = (255, 255, 255)
                if grid[x, y] == 2:
                    color = GOAL_COLOR
                elif grid[x, y] == 1:
                    color = OBSTACLE_COLOR

            pygame.draw.rect(screen, color, rect)  # Draw the cell
            if GRID_SIZE != GRID_SIZE_VERYHIGH:
                pygame.draw.rect(screen, (200, 200, 200), rect, 1)  # Draw the grid lines
    return goal

def draw_path(path):
    """Draws the planned path."""
    for i in range(len(path) - 1):
        pygame.draw.line(screen, PATH_COLOR, path[i], path[i + 1], 5)


def draw_path_along_grid_in_the_center_of_cell(path):
    """Draws the planned path."""
    for i in range(len(path) - 1):
        pygame.draw.line(screen, (0,255,255), (path[i][0]*GRID_SIZE+GRID_SIZE//2, path[i][1]*GRID_SIZE+GRID_SIZE//2), (path[i + 1][0]*GRID_SIZE+GRID_SIZE//2, path[i + 1][1]*GRID_SIZE+GRID_SIZE//2), 5)

def draw_car(car):
    """Draws the car as a rectangle and rotates it based on heading angle."""
    pygame.display.set_caption("Move Object with Arrow Keys")
    

    rotated_car = pygame.transform.rotate(carImage,angle = np.rad2deg(car.get_heading_angle()))
    car_rect = rotated_car.get_rect(center=car.get_position())

    front_wheel = pygame.transform.rotate(wheelImage,angle = np.rad2deg(car.get_heading_angle()+car.get_steering_angle()))
    front_wheel_rect = front_wheel.get_rect(center=translate_position(car.get_position(), car.get_heading_angle(), carImage.get_width()/2.0))
    
    rear_wheel = pygame.transform.rotate(wheelImage,angle = np.rad2deg(car.get_heading_angle()))
    rear_wheel_rect = rear_wheel.get_rect(center=translate_position(car.get_position(), car.get_heading_angle(), -carImage.get_width()/2.0))
    

    screen.blit(rotated_car, car_rect.topleft)
    screen.blit(front_wheel, front_wheel_rect.topleft)
    screen.blit(rear_wheel, rear_wheel_rect.topleft)

def draw_algorithm(algo):
    algoImage = None
    if algo == "BFS":
        algoImage = bfsImage
    if algo == "DFS":
        algoImage = dfsImage
    if algo == "A*":
        algoImage = astarImage

    algoImageRect = algoImage.get_rect(center=car.get_position())

    screen.blit(algoImage, algoImageRect.topleft)
    

# Main simulation loop
running = True
searched_goal = (-1,-1)
done_path = None
bfs_controller = BFS(grid)
dfs_controller = DFS(grid)
algo = "BFS"

def print_algo(algo):
    print("\n\n#######################")
    print(algo)

def print_grid_size(grid):
    print(str(len(grid)) + "x" + str(len(grid[0])))

sys.setrecursionlimit(50000) 
print(sys.getrecursionlimit())
while running:
    dt = clock.tick(FPS) / 1000.0  # Convert to seconds
    
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        if event.type == pygame.KEYDOWN :
            if event.key == pygame.K_w :    #forward
                car.set_speed_rate(1.0)
            elif event.key == pygame.K_s :  #backward
                car.set_speed_rate(-1.0)
            if event.key == pygame.K_a :    #left
                car.set_steering_rate(1.0)
            elif event.key == pygame.K_d :  #right
                car.set_steering_rate(-1.0)
            elif event.key == pygame.K_1 or event.key == pygame.K_KP1:  #BFS
                if algo != "BFS":
                    algo = "BFS"
                    done_path = None
                    goal = (-1,-1)
                    grid = np.zeros((grid_width, grid_height))
            elif event.key == pygame.K_2 or event.key == pygame.K_KP2:  #DFS
                if algo != "DFS":
                    algo = "DFS"
                    done_path = None
                    goal = (-1,-1)
                    grid = np.zeros((grid_width, grid_height))
            elif event.key == pygame.K_3 or event.key == pygame.K_KP3:  #A*
                if algo != "A*":
                    algo = "A*"
                    done_path = None
                    goal = (-1,-1)
                    grid = np.zeros((grid_width, grid_height))
            elif event.key == pygame.K_5 or event.key == pygame.K_KP5: # small
                if GRID_SIZE != GRID_SIZE_SMALL:
                    grid, grid_width, grid_height, GRID_SIZE = refreshGrid(GRID_SIZE_SMALL)
                    done_path = None
                    goal = (-1,-1)
            elif event.key == pygame.K_7 or event.key == pygame.K_KP7: # medium
                if GRID_SIZE != GRID_SIZE_MEDIUM:
                    grid, grid_width, grid_height, GRID_SIZE = refreshGrid(GRID_SIZE_MEDIUM)
                    done_path = None
                    goal = (-1,-1)
            elif event.key == pygame.K_8 or event.key == pygame.K_KP8: # high
                if GRID_SIZE != GRID_SIZE_HIGH:
                    grid, grid_width, grid_height, GRID_SIZE = refreshGrid(GRID_SIZE_HIGH)
                    done_path = None
                    goal = (-1,-1)
            elif event.key == pygame.K_9 or event.key == pygame.K_KP9: # ultra high
                if GRID_SIZE != GRID_SIZE_VERYHIGH:
                    grid, grid_width, grid_height, GRID_SIZE = refreshGrid(GRID_SIZE_VERYHIGH)
                    done_path = None
                    goal = (-1,-1)
            elif event.key == pygame.K_r: # reset grid
                grid = refreshGrid(GRID_SIZE)[0]
            elif event.key == pygame.K_t and pygame.key.get_mods() & pygame.KMOD_SHIFT: # save to slot 1
                # Saving the grid to a file
                with open('grids/grid1.pkl', 'wb') as f:
                    pickle.dump(grid, f)
            # save to slot number two
            elif event.key == pygame.K_y and pygame.key.get_mods() & pygame.KMOD_SHIFT:
                # Saving the grid to a file
                with open('grids/grid2.pkl', 'wb') as f:
                    pickle.dump(grid, f)

            # save to slot number three
            elif event.key == pygame.K_u and pygame.key.get_mods() & pygame.KMOD_SHIFT:
                # Saving the grid to a file
                with open('grids/grid3.pkl', 'wb') as f:
                    pickle.dump(grid, f)
            
            #save to slot number four
            elif event.key == pygame.K_i and pygame.key.get_mods() & pygame.KMOD_SHIFT:
                # Saving the grid to a file
                with open('grids/grid4.pkl', 'wb') as f:
                    pickle.dump(grid, f)
            
            #save to slot number five
            elif event.key == pygame.K_o and pygame.key.get_mods() & pygame.KMOD_SHIFT:
                # Saving the grid to a file
                with open('grids/grid5.pkl', 'wb') as f:
                    pickle.dump(grid, f)

            # save to slot number six
            elif event.key == pygame.K_p and pygame.key.get_mods() & pygame.KMOD_SHIFT:
                # Saving the grid to a file
                with open('grids/grid6.pkl', 'wb') as f:
                    pickle.dump(grid, f)
            
            # load from slot number one
            elif event.key == pygame.K_t and pygame.key.get_mods() & pygame.KMOD_CTRL:
                #check if file exists
                if not os.path.exists('grids/grid1.pkl'):
                    continue
                # Loading the grid from a file
                with open('grids/grid1.pkl', 'rb') as f:
                    grid = pickle.load(f)
                    grid_width = len(grid)
                    grid_height = len(grid[0])
                    done_path = None
                    goal = (-1,-1)

            
            # load from slot number two
            elif event.key == pygame.K_y and pygame.key.get_mods() & pygame.KMOD_CTRL:
                #check if file exists
                if not os.path.exists('grids/grid2.pkl'):
                    continue
                
                # Loading the grid from a file
                with open('grids/grid2.pkl', 'rb') as f:
                    grid = pickle.load(f)
                    grid_width = len(grid)
                    grid_height = len(grid[0])
                    done_path = None
                    goal = (-1,-1)
            
            # load from slot number three
            elif event.key == pygame.K_u and pygame.key.get_mods() & pygame.KMOD_CTRL:
                # check if file exists
                if not os.path.exists('grids/grid3.pkl'):
                    continue
                # Loading the grid from a file
                with open('grids/grid3.pkl', 'rb') as f:
                    grid = pickle.load(f)
                    grid_width = len(grid)
                    grid_height = len(grid[0])
                    done_path = None
                    goal = (-1,-1)
            
            # load from slot number four
            elif event.key == pygame.K_i and pygame.key.get_mods() & pygame.KMOD_CTRL:
                #check if file exists
                if not os.path.exists('grids/grid4.pkl'):
                    continue
                # Loading the grid from a file
                with open('grids/grid4.pkl', 'rb') as f:
                    grid = pickle.load(f)
                    grid_width = len(grid)
                    grid_height = len(grid[0])
                    done_path = None
                    goal = (-1,-1)
            
            # load from slot number five
            elif event.key == pygame.K_o and pygame.key.get_mods() & pygame.KMOD_CTRL:
                #check if file exists
                if not os.path.exists('grids/grid5.pkl'):
                    continue
                
                # Loading the grid from a file
                with open('grids/grid5.pkl', 'rb') as f:
                    grid = pickle.load(f)
                    grid_width = len(grid)
                    grid_height = len(grid[0])
                    done_path = None
                    goal = (-1,-1)
            
            # load from slot number six
            elif event.key == pygame.K_p and pygame.key.get_mods() & pygame.KMOD_CTRL:
                #check if file exists
                if not os.path.exists('grids/grid6.pkl'):
                    continue
                # Loading the grid from a file
                with open('grids/grid6.pkl', 'rb') as f:
                    grid = pickle.load(f)
                    grid_width = len(grid)
                    grid_height = len(grid[0])
                    done_path = None
                    goal = (-1,-1)

            elif event.key == pygame.K_h and pygame.key.get_mods() & pygame.KMOD_CTRL:
                #check if file exists
                if not os.path.exists('city.pkl'):
                    continue
                # Loading the grid from a file
                with open('city.pkl', 'rb') as f:
                    grid = pickle.load(f)
                    grid_width = len(grid)
                    grid_height = len(grid[0])
                    done_path = None
                    goal = (-1,-1)

            elif event.key == pygame.K_k and event.key != pygame.K_k: #save grid
                # Saving the grid to a file
                with open('grid.pkl', 'wb') as f:
                    pickle.dump(grid, f)
            elif event.key == pygame.K_l and event.key != pygame.K_l: #load grid
                # Loading the grid from a file
                with open('grid.pkl', 'rb') as f:
                    grid = pickle.load(f)
                    grid_width = len(grid)
                    grid_height = len(grid[0])
                    done_path = None
                    goal = (-1,-1)
        elif event.type == pygame.KEYUP :
            if event.key == pygame.K_w or event.key == pygame.K_s :
                car.set_speed_rate(0.0)
            if event.key == pygame.K_a or event.key == pygame.K_d :
                car.set_steering_rate(0)
        elif event.type == pygame.MOUSEBUTTONDOWN:
            pass
        elif event.type == pygame.MOUSEBUTTONUP:  # Handle mouse release
            mouse_x, mouse_y = pygame.mouse.get_pos()  # Get mouse position
            mouse_grid_x, mouse_grid_y = mouse_x // GRID_SIZE, mouse_y // GRID_SIZE  # Get the grid cell under the mouse

            #set mouse grid x,y not to exceed the grid
            mouse_grid_x = max(0, min(mouse_grid_x, grid_width-1))
            mouse_grid_y = max(0, min(mouse_grid_y, grid_height-1))

            # On release, set the cell back to hover color if it was clicked
            if grid[mouse_grid_x, mouse_grid_y] == 1 and not pygame.mouse.get_pressed()[0]:
                pass  # The color will automatically update in `draw_grid()` based on the hover condition


    # Clear screen
    screen.fill((255, 255, 255))  # White background
    
    # Draw the grid and obstacles
    goal = draw_grid(goal)

    # Draw the planned path
    #draw_path(path)

    # Move the car along the path
    car.travel(dt)
    
    if goal != (-1,-1) and searched_goal != goal:
        print_algo(algo)
        print_grid_size(grid)
        searched_goal = goal
        before_running = time.time()
        if algo == "BFS":
            searched_path = bfs_controller.bfs_path(start=(car.get_position()[0]//GRID_SIZE, car.get_position()[1]//GRID_SIZE) ,goal = goal, grid=grid.astype(int).tolist())
        if algo == "DFS":
            searched_path = dfs_controller.dfs_path(start=(car.get_position()[0]//GRID_SIZE, car.get_position()[1]//GRID_SIZE) ,goal = goal, grid=grid.astype(int).tolist())
        if algo == "A*":
            searched_path = a_star(start=(car.get_position()[0]//GRID_SIZE, car.get_position()[1]//GRID_SIZE) ,goal = goal, grid=grid.astype(int).tolist())
        
        after_running = time.time()
        print("Microseconds passed: " + f"{(after_running-before_running)*10**3:.15f}")
        if searched_path:
            done_path = searched_path
            print("Path length: " + str(len(searched_path)) + " cells")
        else:
            print("No path found")
            done_path = None
        
        #d_star_lite(start=(car.get_position()[0]//GRID_SIZE, car.get_position()[1]//GRID_SIZE) ,goal = goal, grid=grid)
    if done_path:
        draw_path_along_grid_in_the_center_of_cell(searched_path)

    x = max(0, min(car.get_position()[0], GRID_SIZE*grid_width))
    y = max(0, min(car.get_position()[1], HEIGHT))
    car.set_position((x, y))
    # Draw the car
    draw_car(car)

    # Write algorithm
    draw_algorithm(algo)

    # Update the display
    pygame.display.flip()

# Quit pygame
pygame.quit()


50000


#######################
BFS
26x20
Microseconds passed: 0.000000000000000
Path length: 11 cells


#######################
BFS
26x20
Microseconds passed: 0.000000000000000
Path length: 12 cells


#######################
BFS
26x20
Microseconds passed: 0.997781753540039
Path length: 13 cells


#######################
BFS
26x20
Microseconds passed: 0.997304916381836
Path length: 14 cells


#######################
BFS
26x20
Microseconds passed: 0.997066497802734
Path length: 14 cells


#######################
BFS
26x20
Microseconds passed: 0.000000000000000
Path length: 14 cells


#######################
BFS
26x20
Microseconds passed: 0.000000000000000
Path length: 13 cells


#######################
BFS
26x20
Microseconds passed: 0.000000000000000
Path length: 12 cells


#######################
BFS
26x20
Microseconds passed: 0.000000000000000
Path length: 11 cells


#######################
BFS
26x20
Microseconds passed: 0.997543334960938
Path length: 9 cells


#######################